In [ ]:
import hvac
import os
import requests

from dotenv import load_dotenv

load_dotenv()

In [ ]:
client = hvac.Client(
    url=os.getenv('VAULT_API'),
    token=os.getenv('VAULT_TOKEN'),
)

# print(client.is_authenticated())

try:
    secret_resp = client.secrets.kv.v2.read_secret_version(
        mount_point='kv', 
        path='slack', 
        raise_on_deleted_version=False
    )
    
    if secret_resp['data'] is not None:
        secret_values = secret_resp['data']['data']
        for secret, value in secret_values.items():
            slack_webhook_url = secret_values['SLACK_WEBHOOK_URL']
    else:
        print("The secret does not exist.")
except hvac.exceptions.InvalidPath:
    print("The path is invalid or the permission is denied.")
except hvac.exceptions.Forbidden:
    print("The permission is denied.")
except hvac.exceptions.VaultError as e:
    print(f"Vault error occurred: {e}")

In [ ]:
url_list = [
    'https://fairbanks.io',
    'https://fairbanks.dev',
    'https://fbnks.io',
    'https://bsord.io',
    'https://f5.news'
]

for url in url_list:
    url_resp = requests.get(url)
    if(url_resp.status_code == 200 or url_resp.status_code == 201):
        #print(f"{url} responded with HTTP 2xx as expected")
        pass
    else:
        print(f"{url} request error -- HTTP {url_resp.status_code}")
        headers = { "Content-Type": "application/json" }
        msg_body = f":warning: {url} failed to load successfully! :warning:"
        body = { "text": msg_body }
        #call a webhook
        response = requests.post(slack_webhook_url, json=body, headers=headers)